In [1]:
from pikan.model_utils import GeneralizedMLP, FourierKAN, PirateNet
from pikan.model_utils import get_mse_loss, get_train_step
from pikan.model_utils import KeyHandler, sobol_sample
from pikan.utils import load_dict_from_file, save_dict_to_file

from pikan.interpolated_funcs import circular_wave_interference

import yaml
import jax.numpy as jnp
import numpy as np
from flax import linen as nn
import optax
from functools import partial
from tqdm import tqdm
import pandas as pd

import jax
import pickle
import os

jax.devices()

[CudaDevice(id=0)]

In [2]:
def l2_error(results, true):
    err = jnp.sum((results - true)**2) / jnp.sum(true**2)
    err = jnp.sqrt(err)
    return err

def get_l2_error(config, variables):
    model = get_model(config)
    learnable_func = get_target_func(experiment)
    
    N = 300
    X_1 = jnp.linspace(experiment["X_MIN"], experiment["X_MAX"], N)
    X_2 = jnp.linspace(experiment["Y_MIN"], experiment["Y_MAX"], N)
    X_1, X_2 = jnp.meshgrid(X_1, X_2, indexing='ij')
    coords = jnp.stack([X_1.flatten(), X_2.flatten()], axis=1)

    y = learnable_func(coords).reshape(-1,1)

    if config["MODEL"] == "KAN":
        y_hat = model.apply(variables, coords)
    else:
        y_hat = model.apply({"params": variables["params"]}, coords)
        
    err = l2_error(y_hat, y)
    
    return err

def sum_params(data, verbose=False):
    total = 0
    if isinstance(data, type(jnp.array([]))):  # If the current node is a leaf array
        return len(data.reshape(-1))
    elif isinstance(data, dict):  # If the current node is a dictionary
        for key, value in data.items():
            if verbose:
                print(f"Processing key: {key}")  # Print the current key
            branch_total = sum_params(value)  # Compute the total for this subbranch
            if verbose:
                print(f"Total parameters in subbranch '{key}': {branch_total}")
            total += branch_total
    
    return total

In [3]:
def get_model(config):
    if config["MODEL"] == "MLP":
        return GeneralizedMLP(
            kernel_init=nn.initializers.glorot_normal(),
            num_input=config['N_INPUT'],
            num_output=1,
            use_fourier_feats=config['FourierFeatures'],
            layer_sizes=config['layers']
        )
    if config["MODEL"] == "KAN":
        return FourierKAN(
            kernel_init=nn.initializers.glorot_normal(),
            num_input=config['N_INPUT'],
            num_output=1,
            use_fourier_feats=config['FourierFeatures'],
            layer_sizes=config['layers']
        )
    if config["MODEL"] == "PIRATE":
        return PirateNet(
            kernel_init=nn.initializers.glorot_normal(),
            num_input=config['N_INPUT'],
            num_output=1,
            layer_sizes=config['layers'] # first is fourier
        )
    
def get_target_func(config):
    if experiment["learnable_func"] == "circular_wave_interference":
        learnable_func = circular_wave_interference

    learnable_func = partial(learnable_func, FREQ=experiment["FREQ"])
    return learnable_func

def sample_collocs(config):
    collocs = jnp.array(sobol_sample(np.array([config["X_MIN"],config["Y_MIN"]]), 
                                     np.array([config["X_MAX"],config["Y_MAX"]]), config["BS"]))
    return collocs

def train_model(config):    
    collocs = sample_collocs(config)

    model = get_model(config)
    variables = model.init(keygen.key(), collocs)
    loss_fn = get_mse_loss(model, MODEL=config["MODEL"])
    
    # Define a cosine decay learning rate schedule
    schedule_fn = optax.cosine_decay_schedule(
        init_value=1e-2,       # Initial learning rate
        decay_steps=config["EPOCHS"],  # Total number of decay steps
        alpha=1e-3             # Final learning rate multiplier
    )
    optimizer = optax.adamw(learning_rate=schedule_fn, weight_decay=1e-4)
    opt_state = optimizer.init(variables['params'])
    train_step = get_train_step(model, optimizer, loss_fn)

    learnable_func = get_target_func(config)

    if config["MODEL"] != "KAN":
        variables["state"] = []

    # train always on same colloc points
    collocs = sample_collocs(experiment)
    losses = []

    loc_w = jnp.array([])
    for i in (pbar:= tqdm(range(experiment["EPOCHS"]))):
        params, state = variables['params'], variables['state']
        y = learnable_func(collocs).reshape(-1,1)
        params, opt_state, loss, loc_w = train_step(params, collocs, y,
                                                    opt_state, state, loc_w)
        variables = {'params': params, 'state':state}

        losses.append(loss)

        if i % 50 == 0: # dont waste a lot of time printing
            pbar.set_description(f"Loss {loss: .8f}")

    return variables

In [4]:
filename = "increase_freq_KAN_Fourier"
# filename = "increase_freq_PIRATE"

with open(f"yaml_configs/{filename}.yaml", 'r') as file:
    config = yaml.safe_load(file)

keygen = KeyHandler(0)
config["experiments"].keys()

dict_keys(['increase_freq_1', 'increase_freq_2', 'increase_freq_3', 'increase_freq_4', 'increase_freq_5', 'increase_freq_6', 'increase_freq_7', 'increase_freq_8', 'increase_freq_9', 'increase_freq_10'])

In [5]:
for exp_key in config["experiments"].keys():
    experiment = config["experiments"][exp_key]

    collocs = sample_collocs(experiment)
    model = get_model(experiment)
    variables = model.init(keygen.key(), collocs)

    print(exp_key)
    print(sum_params(variables["params"], verbose=False))

increase_freq_1
12513
increase_freq_2
12513
increase_freq_3
12513
increase_freq_4
12513
increase_freq_5
12513
increase_freq_6
12513
increase_freq_7
12513
increase_freq_8
12513
increase_freq_9
12513
increase_freq_10
12513


In [6]:
import time

elapsed_times = []
for exp_key in config["experiments"].keys():
    experiment = config["experiments"][exp_key]
    start_time = time.time()
    variables = train_model(experiment)
    end_time = time.time()
    
    save_dict_to_file(variables, f"results/models/{filename}", f"{exp_key}")
    elapsed_times.append(end_time - start_time)

print("Elapsed times for each iteration:", elapsed_times)

Loss  0.01053457: 100%|█████████████████████| 2500/2500 [00:46<00:00, 54.03it/s]


Elapsed times for each iteration: [48.187947273254395, 47.602500677108765, 47.638744592666626, 47.74214148521423, 47.62265372276306, 47.63312029838562, 46.307276010513306, 47.66452622413635, 46.398332834243774, 46.3160195350647]


In [7]:
import pandas as pd

df = pd.DataFrame(config["experiments"]).T
df["params"] = -1
df["L2%"] = -1

for idx, exp_key in enumerate(config["experiments"].keys()):
    experiment = config["experiments"][exp_key]
    
    variables = load_dict_from_file(f"results/models/{filename}/{exp_key}")
    l2_err = get_l2_error(experiment, variables)

    df.loc[exp_key, "params"] = sum_params(variables["params"], verbose=False)
    df.loc[exp_key, "L2%"] = l2_err * 100
    df['seconds'] = elapsed_times
    
    print(f"Results from {exp_key}:")
    print(f"L2 {l2_err*100:.4f}%")
    print(f"#params {df.iloc[idx]['params']}")
    
df.to_csv(f"results/csvs/{filename}")

/tmp/ipykernel_10501/2905213090.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.3995928764343262' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[exp_key, "L2%"] = l2_err * 100


Results from increase_freq_1:
L2 1.3996%
#params 12513
Results from increase_freq_2:
L2 0.8751%
#params 12513
Results from increase_freq_3:
L2 0.9127%
#params 12513
Results from increase_freq_4:
L2 1.3593%
#params 12513
Results from increase_freq_5:
L2 1.6934%
#params 12513
Results from increase_freq_6:
L2 2.3574%
#params 12513
Results from increase_freq_7:
L2 2.4075%
#params 12513
Results from increase_freq_8:
L2 3.8815%
#params 12513
Results from increase_freq_9:
L2 4.8579%
#params 12513
Results from increase_freq_10:
L2 9.4300%
#params 12513


In [8]:
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
from matplotlib import cm
import os

def plot_true_approx(config, variables, save_folder, exp_key):
    model = get_model(config)
    learnable_func = get_target_func(config)  # Fixed to use 'config' instead of 'experiment'
    
    N = 300
    X_1 = jnp.linspace(config["X_MIN"], config["X_MAX"], N)  # Fixed to use 'config'
    X_2 = jnp.linspace(config["Y_MIN"], config["Y_MAX"], N)  # Fixed to use 'config'
    X_1, X_2 = jnp.meshgrid(X_1, X_2, indexing='ij')
    coords = jnp.stack([X_1.flatten(), X_2.flatten()], axis=1)

    y = learnable_func(coords).reshape(-1, 1)
    if config["MODEL"] == "KAN":
        y_hat = model.apply(variables, coords)
    else:
        y_hat = model.apply({"params": variables["params"]}, coords)

    y_hat = y_hat.reshape(-1, 1)

    # Compute the absolute error
    abs_error = jnp.abs(y - y_hat).reshape(-1, 1)

    # Create a figure and axis grid for the 3 subplots
    fig = plt.figure(figsize=(18, 6))

    # Plotting the approximated function (y_hat) on the left using the plasma colormap
    ax1 = fig.add_subplot(1, 3, 1, projection='3d')
    ax1.plot_trisurf(coords[:, 0], coords[:, 1], y_hat.flatten(), cmap=cm.plasma)
    ax1.set_title('Approximated Function (y_hat)')
    ax1.set_xlabel('X1')
    ax1.set_ylabel('X2')
    ax1.set_zlabel('y_hat')

    # Plotting the true function (y) in the middle
    ax2 = fig.add_subplot(1, 3, 2, projection='3d')
    ax2.plot_trisurf(coords[:, 0], coords[:, 1], y.flatten(), cmap='viridis')
    ax2.set_title('True Function (y)')
    ax2.set_xlabel('X1')
    ax2.set_ylabel('X2')
    ax2.set_zlabel('y')

    # Plotting the absolute error on the right
    ax3 = fig.add_subplot(1, 3, 3, projection='3d')
    ax3.plot_trisurf(coords[:, 0], coords[:, 1], abs_error.flatten(), cmap='inferno')
    ax3.set_title('Absolute Error')
    ax3.set_xlabel('X1')
    ax3.set_ylabel('X2')
    ax3.set_zlabel('Error')

    # Adjust layout to ensure all plots fit well within the figure
    plt.tight_layout(pad=3.0)

    # Saving the figure
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    save_path = os.path.join(save_folder, f'{exp_key}_plot.png')

    # Save the plot to the specified path
    fig.savefig(save_path)

    # Explicitly close the plot to prevent it from showing
    plt.close(fig)

In [13]:
from concurrent.futures import ThreadPoolExecutor

def process_experiment(exp_key):
    try:
        experiment = config["experiments"][exp_key]
        print(f"Results from {exp_key}:")
        variables = load_dict_from_file(f'results/models/{filename}/{exp_key}')
        plot_true_approx(experiment, variables, f"results/visuals/{filename}", exp_key)
    except Exception as e:
        print(f"Error processing {exp_key}: {e}")

import matplotlib
matplotlib.use('Agg')

with ThreadPoolExecutor(max_workers=1) as executor:
    # Ensure we exhaust the map generator
    list(executor.map(process_experiment, config["experiments"].keys()))

Results from increase_freq_1:
Results from increase_freq_2:
Results from increase_freq_3:
Results from increase_freq_4:
Results from increase_freq_5:
Results from increase_freq_6:
Results from increase_freq_7:
Results from increase_freq_8:
Results from increase_freq_9:
Results from increase_freq_10:


In [18]:
display(pd.read_csv('results/csvs/increase_freq_fourier').tail(3))

display(pd.read_csv('results/csvs/increase_freq_KAN_Fourier').tail(3))

,Unnamed: 0,BS,EPOCHS,FREQ,learnable_func,FourierFeatures,MODEL,layers,N_INPUT,X_MIN,X_MAX,Y_MIN,Y_MAX,params,L2%,seconds
7,increase_freq_8,16384,2500,8,circular_wave_interference,True,MLP,"[64, 64, 64, 64]",2,-1,1,-1,1,12641,0.858057,1.696438
8,increase_freq_9,16384,2500,9,circular_wave_interference,True,MLP,"[64, 64, 64, 64]",2,-1,1,-1,1,12641,0.959238,1.730715
9,increase_freq_10,16384,2500,10,circular_wave_interference,True,MLP,"[64, 64, 64, 64]",2,-1,1,-1,1,12641,1.025996,1.709908


,Unnamed: 0,BS,EPOCHS,FREQ,learnable_func,FourierFeatures,MODEL,layers,N_INPUT,X_MIN,X_MAX,Y_MIN,Y_MAX,params,L2%,seconds
7,increase_freq_8,16384,2500,8,circular_wave_interference,True,KAN,"[32, 32, 16]",2,-1,1,-1,1,12513,3.881505,47.664526
8,increase_freq_9,16384,2500,9,circular_wave_interference,True,KAN,"[32, 32, 16]",2,-1,1,-1,1,12513,4.857927,46.398333
9,increase_freq_10,16384,2500,10,circular_wave_interference,True,KAN,"[32, 32, 16]",2,-1,1,-1,1,12513,9.430018,46.316020
